__sentiment analysis__

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
import numpy as np
import pandas as pd
import os
import re

In [67]:


reviews_train = []
for line in open('/Users/ndaru/Documents/rai_pwdk/self_learn/daily_challenge/#10_day/Sentiment-Analysis-of-Movie-Reviews-using-CNN-master/train.txt', 'r'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('/Users/ndaru/Documents/rai_pwdk/self_learn/daily_challenge/#10_day/Sentiment-Analysis-of-Movie-Reviews-using-CNN-master/test1.txt', 'r'):
    reviews_test.append(line.strip())
for line in open('/Users/ndaru/Documents/rai_pwdk/self_learn/daily_challenge/#10_day/Sentiment-Analysis-of-Movie-Reviews-using-CNN-master/test2.txt', 'r'):
    reviews_test.append(line.strip())
for line in open('/Users/ndaru/Documents/rai_pwdk/self_learn/daily_challenge/#10_day/Sentiment-Analysis-of-Movie-Reviews-using-CNN-master/test3.txt', 'r'):
    reviews_test.append(line.strip())

In [11]:
reviews_train[10]

"10::even when he 's not at his most critically insightful , godard can still be smarter than any 50 other filmmakers still at work::1"

In [69]:
display(print(len(reviews_train)))
print(len(reviews_test))

6001


None

3003


In [4]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [12]:
reviews_train_clean[10]

'even when he s not at his most critically insightful  godard can still be smarter than any  other filmmakers still at work'

In [51]:
print(len(reviews_train_clean))

6001


In [63]:
cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

target = [1 if i < 5000 else 0 for i in range(6001)]

In [49]:
# train_sizes, train_scores, test_scores = learning_curve(LogisticRegression(), X, target, cv = 5, n_jobs = 4, train_sizes = 0.75,scoring="accuracy",  shuffle=True)


In [64]:
X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

In [66]:
for c in [0.01, 0.025, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c, max_iter=150)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s \t: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01 	: 0.832778147901399
Accuracy for C=0.025 	: 0.832778147901399
Accuracy for C=0.25 	: 0.8287808127914723
Accuracy for C=0.5 	: 0.82145236508994
Accuracy for C=1 	: 0.8087941372418388


In [60]:
final_model = LogisticRegression(C=0.01)
final_model.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_model.predict(X_test)))

ValueError: Found input variables with inconsistent numbers of samples: [6001, 1001]

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
    
#     ('excellent', 0.9288812418118644)
#     ('perfect', 0.7934641227980576)
#     ('great', 0.675040909917553)
#     ('amazing', 0.6160398142631545)
#     ('superb', 0.6063967799425831)
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)
    
#     ('worst', -1.367978497228895)
#     ('waste', -1.1684451288279047)
#     ('awful', -1.0277001734353677)
#     ('poorly', -0.8748317895742782)
#     ('boring', -0.8587249740682945)